In [4]:
import pandas as pd
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from requests import get
from tika import parser
import urllib.request

ModuleNotFoundError: No module named 'pdfminer'

In [ ]:
def download_save_img(url, file_name):
    urllib.request.urlretrieve(url, file_name)

def download_pdf(url, file_name):
    with open(file_name, "wb") as file:
        response = get(url)
        file.write(response.content)
        
def read_and_write(file_name, text_name):
    with open(text_name, "w", encoding='utf8') as writer:
        parsed = parser.from_file(file_name)
        text = parsed['content']
        lines = text.split('\n')
        full_text = []
        for line in lines:
            if line == '':
                continue
            else:
                writer.write(line.replace('�', ' ')+'\n')
                
def read_and_return(file_name):
    parsed = parser.from_file(file_name)
    text = parsed['content']
    lines = text.split('\n')
    full_text = ""
    for line in lines:
        if line == '':
            continue
        else:
            full_text+=(line.replace('�', ' ')+'\n')
    return full_text

In [ ]:
data_detail = pd.read_excel('csv/pill_details.xls')
data_detail = data_detail[['품목일련번호', '허가일자', '원료성분', '마약류분류', '완제원료구분', '포장단위', '보험코드', '전문일반', '총량', '취소상태',
       '변경내용', '등록자id', '첨부문서', '제심사기간', '효능효과', '저장방법', '신약여부', '유효기간',
       '용법용량', '첨가제명', '성상', '재심사대상', '허가및신고구분',
       '취소일자', '표준코드', '주의사항', '주성분명']]

data_list = pd.read_csv('csv/pill_list.csv')
data_list = data_list[['품목일련번호', '품목명', '업소일련번호', '업소명', '큰제품이미지', '표시앞', '표시뒤', '의약품제형',
       '색상앞', '색상뒤', '분할선앞', '분할선뒤', '크기장축', '크기단축', '크기두께', '이미지생성일자(약학정보원)',
       '분류번호', '분류명', '전문일반구분', '품목허가일자', '제형코드명', '표기내용앞', '표기내용뒤', '표기이미지앞',
       '표기이미지뒤', '표기코드앞', '표기코드뒤', '변경일자']]

In [ ]:
new_Data = pd.merge(data_list, data_detail, on='품목일련번호')

In [ ]:
img = '큰제품이미지'
eff = '효능효과'
use = '용법용량'
war = '주의사항'

In [ ]:
row_num = len(new_Data.index)
print("total row number:",row_num)

for idx in range(5):
    pill_id = new_Data.loc[idx]['품목일련번호']
    print(idx,"Pill ID: "+str(pill_id)+"-"*20)
    pill_img = new_Data.loc[idx][img]
    pill_eff = new_Data.loc[idx][eff]
    pill_use = new_Data.loc[idx][use]
    pill_war = new_Data.loc[idx][war]
    
    ## 큰제품이미지
    file_name = str(pill_id)+'_img'
    download_save_img(pill_img, 'img/'+file_name+'.jpg')
    new_Data.iloc[idx,4] = file_name+'.jpg'
    print("image Done.")
    
    ## 효능효과
    file_name = str(pill_id)+'_effect'
    download_pdf(pill_eff, 'pdf/'+file_name+'.pdf')
    text = read_and_return('pdf/'+file_name+'.pdf')
    new_Data.iloc[idx,41] = text
    print(new_Data.loc[idx][eff])
    print("effect Done.")
    
    ## 용법용량
    file_name = str(pill_id)+'_usage'
    download_pdf(pill_use, 'pdf/'+file_name+'.pdf')
    text = read_and_return('pdf/'+file_name+'.pdf')
    new_Data.iloc[idx,45] = text
    print(new_Data.loc[idx][use])
    print("usage Done.")
    
    ## 주의사항
    file_name = str(pill_id)+'_warning'
    download_pdf(pill_war, 'pdf/'+file_name+'.pdf')
    text = read_and_return('pdf/'+file_name+'.pdf')
    new_Data.iloc[idx,52] = text
    print(new_Data.loc[idx][war])
    print("warning Done.")

In [ ]:
new_Data.to_excel("csv/pill_information.xlsx")

new_Data.to_csv("csv/pill_information.csv", "w")

In [ ]:
def download_save_img(url, file_name):
    urllib.request.urlretrieve(url, file_name)